In [1]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"
%matplotlib inline
plt.style.use('ggplot')

In [2]:
# Jupyter Specifics
from IPython.display import HTML
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout

style = {'description_width': '100px'}
slider_layout = Layout(width='99%')

In [3]:
def ode_model(z, t, beta, sigma):
    """
    Reference https://www.idmod.org/docs/hiv/model-seir.html
    """
    S, I, R = z
    N = S + I + R
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - sigma * I
    dRdt = sigma * I
    return [dSdt, dIdt, dRdt]

In [4]:
def ode_solver(t, initial_conditions, params):
    initI, initR, initN = initial_conditions
    beta, sigma = params
    initS = initN - (initI + initR)
    res = odeint(ode_model, [initS, initI, initR], t, args=(beta, sigma))
    return res

In [5]:
# ref: https://www.medrxiv.org/content/10.1101/2020.04.01.20049825v1.full.pdf
initN = 1380000000
# S0 = 966000000
initE = 1
initI = 1
initR = 0
sigma = 1/5.2
gamma = 1/2.9
R0 = 4
beta = R0 * gamma
days = 150

In [6]:
from sklearn.metrics import r2_score
from week7_cascade.dataloader import korea_df

from week7_cascade.dataloader import us_df

from week7_cascade.dataloader import uk_df

from week7_cascade.dataloader import global_df

real_range = [0, 300]

def main(initI, initR, initN, beta, sigma, days):
    initial_conditions = [initI, initR, initN]
    params = [beta, sigma]
    tspan = np.arange(0, days, 1)
    sol = ode_solver(tspan, initial_conditions, params)
    S, I, R = sol[:, 0], sol[:, 1], sol[:, 2]
    
    # Create traces
    fig = go.Figure()
#     fig.add_trace(go.Scatter(x=tspan, y=S, mode='lines+markers', name='Susceptible'))
#     fig.add_trace(go.Scatter(x=tspan, y=E, mode='lines+markers', name='Exposed'))
    fig.add_trace(go.Scatter(x=tspan, y=I, mode='lines+markers', name='Infected'))
#     fig.add_trace(go.Scatter(x=tspan, y=R, mode='lines+markers',name='Recovered'))

    country = "global"

    country_df = {
        "korea": korea_df,
        "us": us_df,
        "uk": uk_df,
        "global": global_df
    }[country]
    real_data = (country_df["Confirmed"] - (country_df["Recovered"] + country_df["Deaths"])).values
#     real_data = country_df["Confirmed"].values
    real_data = real_data[real_range[0]: real_range[1]]
    base = real_data[0]
    real_data -= base
    fig.add_trace(go.Scatter(x=tspan, y=real_data, mode='lines+markers', name="real"))
    if days <= 30:
        step = 1
    elif days <= 90:
        step = 7
    else:
        step = 30

    r2 = r2_score(real_data, I)

    # Edit the layout
    fig.update_layout(title=f'SIR {country} (r2: {round(r2, 3)}, date: {real_range}, N: {initN}, b: {beta}, s: {sigma}',
                       xaxis_title='Day',
                       yaxis_title='Counts',
                       title_x=0.5,
                      width=900, height=600
                     )
    fig.update_xaxes(tickangle=-90, tickformat = None, tickmode='array', tickvals=np.arange(0, days + 1, step))
    if not os.path.exists("images"):
        os.mkdir("images")
    fig.write_image("images/sir_simulation.png")
    fig.show()

In [9]:

interact(main, initI=IntSlider(min=1, max=100000, step=10, value=initI, description='initI', style=style, layout=slider_layout),
               initR=IntSlider(min=0, max=100000, step=10, value=initR, description='initR', style=style, layout=slider_layout),
               initN=IntSlider(min=0, max=13800000, step=100, value=initN, description='initN', style=style, layout=slider_layout),
               beta=FloatSlider(min=0, max=4, step=0.01, value=beta, description='Infection rate', style=style, layout=slider_layout),
               sigma=FloatSlider(min=0, max=4, step=0.01, value=sigma, description='Incubation rate', style=style, layout=slider_layout),
               days=IntSlider(min=1, max=10000, step=1, value=real_range[1] - real_range[0], description='Days', style=style, layout=slider_layout)
        );

**References:**<br>
1. SEIR and SEIRS Model https://www.idmod.org/docs/hiv/model-seir.html<br>
2. Compartmental models in epidemiology https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SEIR_model<br>
3. Solve Differential Equations in Python https://www.youtube.com/watch?v=VV3BnroVjZo<br>
4. Computational Statistics in Python https://people.duke.edu/~ccc14/sta-663/CalibratingODEs.html<br>
5. Ordinary Differential Equations (ODE) with Python and Jupyter https://elc.github.io/posts/ordinary-differential-equations-with-python/<br>
6. SEIRS+ Model https://github.com/ryansmcgee/seirsplus<br>
7. Stack Overflow https://stackoverflow.com/questions/40753159/why-is-scipy-minimize-ignoring-my-constraints<br>
8. Lotka–Volterra equations https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations<br>
9. SEIR and Regression Model based COVID-19 outbreak predictions in India https://www.medrxiv.org/content/10.1101/2020.04.01.20049825v1.full.pdf<br>

A simulator built with RShiny which provides many more parameters https://alhill.shinyapps.io/COVID19seir/